In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

## Setup session
sagemaker_session = sagemaker.Session()

#IAM role
role = get_execution_role()

# region
region = sagemaker_session.boto_session.region_name

In [ ]:
training_data_uri="s3://sagemaker-model-scripts/tensorflow/mnist"
training_data_uri="s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)
print(training_data_uri)

In [ ]:
from sagemaker.tensorflow import TensorFlow

# Tensorflow model config
mnist_estimator2 = Tensorflow(
    entry_point = "mnist.py",
    role = role,
    instance_count=2,
    instance_type="ml.m4.2xlarge",
    framework_version="2.1",
    py_version="py2",
    distribution={"parameter_server": {"enabled": True}}
)

In [ ]:
# training the model
mnist_estimator2.fit(training_data_uri)

# deploying a tensorflow model files using EI
predictor = mnist_estimator2.deploy(
      initial_instance_count=1, # ec2 inital count
      instance_type='ml.m4.xlarge', # ec2 instance type
      accelerator_type='ml.eia2.medium' # accelerator type
)

In [ ]:
# prediction
import numpy as np
random_input = np.random.rand(1, 1, 3, 3)

# predict for a sample input
prediction = predictor.predict({"input": random_input.tolist()})

# print prediction
print(prediction)
print(prediction.endpoint)

import sagemaker

# delete the endpoint
predictor.delete_endpoint()